# Using BIP322 with a P2SH MultiSig Locking Script

## This notebook explores PSBT to create a multisig BIP0322


## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and [Digital Contract Design](https://contract.design/). Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

## Notebook Steps

1. Generate a tagged hash of the message to sign (verify against test vector)
2. Define 3 signing parties
3. Create 2-of-3 MultiSig Pay-to-Script Hash
4. Create the virtual to_spend transaction
4. Create unsigned PSBT virtual to_sign transaction
5. Sign the PSBT to Satisfy MultiSig
6. Finalize the PSBT
7. Generated the Bitcoin Tx from PSBT
8. Serialize and encode to_sign Transaction as BIP322 Signature
9. Verify the signature is a valid BIP322 signature on the message and address

## 1. Generate a Tagged Hash of Message to Sign

In [1]:
from buidl.hash import tagged_hash

In [2]:
message = b"Hello World"

# The tag defined in BIP0322 that should be used
tag = b"BIP0322-signed-message"

tagged_hash = tagged_hash(tag,message)

## 2. Define 3 Signing Parties

Note: These would normally be distinct entities who would not have knowledge of each others private keys.

### 2.1. Generate a Root HDPrivateKey from a Seed for each actor

In [3]:
from buidl.ecc import PrivateKey
from buidl.hd import HDPrivateKey

seed_a = b"willssecretseed"
hd_root_a= HDPrivateKey.from_seed(seed_a)

seed_b = b"signer2secretseed"
hd_root_b= HDPrivateKey.from_seed(seed_b)

seed_c = b"signer3secretseed"
hd_root_c= HDPrivateKey.from_seed(seed_c)

### 2.2. Take the fingerprint of the root HDPublicKey

Note: A fingerprint is the hash160's first 4 bytes

In [4]:
xfp_a = hd_root_a.fingerprint().hex()
xfp_b = hd_root_b.fingerprint().hex()
xfp_c = hd_root_c.fingerprint().hex()

print(xfp_a, xfp_b, xfp_c)

d131705e 80e54fd9 e0af9290


### 2.3 Define a HD path to traverse from the root to get a public/private key pair to be used for signing

Note: in this instance we have the same path used for all actors. This does not need to be the case.

In [5]:
# Typically root path traversing handled by wallet software
root_path = "m/45'/0/0/0"

hd_priv_a = hd_root_a.traverse(root_path)
hd_priv_b = hd_root_b.traverse(root_path)
hd_priv_c = hd_root_c.traverse(root_path)



In [6]:
# S256Point representing public key for each actor at a specific HD Path

pubkey_a = hd_priv_a.pub.point
pubkey_b = hd_priv_b.pub.point
pubkey_c = hd_priv_c.pub.point

### 2.4 Create xPub representations of public key to be used in multisig

In [7]:
xpub_a = hd_priv_a.xpub()
xpub_b = hd_priv_b.xpub()
xpub_c = hd_priv_c.xpub()


## 3. Create 2-of-3 MultiSig Pay-to-Script Hash

Script commands are `2 <Public Key A> <Public Key B> <Public Key C> 3 CHECKMULTISIG`


In [8]:
from buidl.script import RedeemScript

redeem_script_to_use = RedeemScript.create_p2sh_multisig(
            quorum_m=2,
            pubkey_hexes=[pubkey_a.sec().hex(),pubkey_b.sec().hex(),pubkey_c.sec().hex()],
            sort_keys=False,
        )

# commands = [2, sk_a.point.sec(), sk_b.point.sec(), sk_c.point.sec(), 3, 174]

# redeem_script = RedeemScript(commands)

script_pubkey = redeem_script_to_use.script_pubkey()
p2sh_address = redeem_script_to_use.address()

In [9]:
print(redeem_script_to_use)
print(script_pubkey)
print(p2sh_address)

OP_2 02e7d79fa51fa52a6cc4e7dfbd53f9bcbe3c1726f1166bef2a7ecaee24548096e1 03ecb9100910c957b5003ff8529226effb28d510a0dbec15f08949e914aa1c6000 03058bbc8764b23c4b007a36d29dcdf00af8c3dbad72a5888e75380f15d3505378 OP_3 OP_CHECKMULTISIG 
OP_HASH160 78cf5a9a660f330b60396d42b050f9db89e00276 OP_EQUAL 
3ChoQmPrnmuuyW3tebfrMqiZgfXYRvXb99


### 3.* Messing around with P2WSH

Note: currently the helper function `create_multisig_psbt` does not support anything other than `p2sh` addresses.

It SHOULD be possible to instatiate a PSBT without the helper function that is a p2wsh multisig. Similarly with p2tr multisig.

It WOULD be ideal to extend the helper function to support these additional script types.

In [10]:
from buidl.script import WitnessScript

witness_script = WitnessScript.convert(redeem_script_to_use.raw_serialize())
p2wsh_script_pubkey = witness_script.script_pubkey()
p2wsh_address = witness_script.address()
# segwit_script_pubkey.address()

## 4. Create the virtual to_spend transaction locked with the p2sh_multisig

BIP 0322 defines the structure of this transaction as follows:

```python
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

The message_challenge is the above 2-3 p2sh scriptPubKey

In [11]:
from buidl.tx import Tx, TxIn, TxOut
from buidl.script import Script,P2WPKHScriptPubKey
from buidl.helper import big_endian_to_int

### 4.1 Create the single transaction input

Note: the script commands `[0x00, 0x20, message_hash]`
- `0` is OP_0, which pushes a 0 onto the stack
- `32` is PUSH32 which pushes the next 32 bytes onto the stack
- `message_hash` is that next 32 bytes. Which is a tagged_hash of the message being signed.

**However, the implementation only requires commands [0, message_hash] be input. PUSH32 is implied by the size of the message_hash and added by the underlying library.**


In [12]:
# Not a valid Tx hash. Will never be spendable on any BTC network.
prevout_hash = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# prevout.n
prevout_index = big_endian_to_int(bytes.fromhex('FFFFFFFF'))
sequence = 0

# Byte array of message hash
message_hash = tagged_hash

# Note this used to be: commands = [0, 32, message_hash] (as per BIP)
# It appears the PUSH32 is implied and added by the size of the message added to the stack
commands = [0, message_hash]
scriptSig = Script(commands)
# Create Tx Input
tx_in = TxIn(prevout_hash,prevout_index,scriptSig,sequence)

### 4.2 Create the single transaction output

In [13]:
# Value of tx output
value = 0

# Convert segwit address to script_pubkey
script_pubkey = script_pubkey

print(script_pubkey)
tx_out = TxOut(value,script_pubkey)

OP_HASH160 78cf5a9a660f330b60396d42b050f9db89e00276 OP_EQUAL 


### 4.3 Create transaction using tx_in and tx_out

In [14]:
# create transaction
version=0
tx_inputs = [tx_in]
tx_outputs = [tx_out]
locktime=0
network="mainnet"

# Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
segwit=True

virtual_to_spend_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)

In [15]:
virtual_to_spend_tx


tx: f7fd427441201342a93a9252b4e95458acca7fb12aef4dd5145d22c0cd1af6a6
version: 0
locktime: 0
tx_ins:
0000000000000000000000000000000000000000000000000000000000000000:4294967295
tx_outs:
0:OP_HASH160 78cf5a9a660f330b60396d42b050f9db89e00276 OP_EQUAL 

## 5. Create PSBT virtual to_sign transaction

The PSBT data structure is defined in [BIP174](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki)

The structure as defined in BIP 0322:

```python
    nVersion = 0 or (FULL format only) as appropriate (e.g. 2, for time locks)
    nLockTime = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].prevout.hash = to_spend.txid
    vin[0].prevout.n = 0
    vin[0].nSequence = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].scriptWitness = message_signature
    vout[0].nValue = 0
    vout[0].scriptPubKey = OP_RETURN
```



### 5.1 Define Public Key Records of participants in MultiSig

These values were produced in Step 2. 

Note: This requires some coordination between MultiSig participants, at least one party must know this information for all participants to be able to produce a PSBT.

Each record contains:
* A fingerprint
* An xpub
* A root path through HD key

In [16]:
pubkey_records = [
    [xfp_a,xpub_a,root_path],
    [xfp_b,xpub_b,root_path],
    [xfp_c,xpub_c,root_path]
]


In [17]:
pubkey_records

[['d131705e',
  'xpub6EyAEVH13bj6wGsMw79nTAeRhKm7d1LV3NzsUosqocvTuNi9MMk6vGNnWvFN4b1tVRjumnkawQUxMTsySnq74PcEcTnqXVT92t3wq9ztUsM',
  "m/45'/0/0/0"],
 ['80e54fd9',
  'xpub6EEcNjCVVKJedKvLsWJD8zNDuNXfzMxHZc4R71qwVPBjh3ByZYSeDsCQnVcpjMhWzi9iuqVaBYX2WiucFyXUxc7oYzeiAwpbnJq1jDimhQN',
  "m/45'/0/0/0"],
 ['e0af9290',
  'xpub6DhSeTsokQiYjDHcqZwE8iTrj9BYuDoLeXJPhAieoNuTt3sr74kELp4CDMJ6s3RRLciTs3goxP89yJadMSDsazHQHb7sVB9uQyoxUAkqMvE',
  "m/45'/0/0/0"]]

### 5.2 Define Input Dicts

This is used in the helper function

Structure taken from buidl-python `test_psbt_helper.py` file

```json
            "input_dicts": [
                {
                    "quorum_m": 1,
                    "path_list": [
                        # (xfp, root_path)
                        ("838f3ff9", "m/45'/0/0/0"),
                        ("e0c595c5", "m/45'/0/0/0"),
                    ],
                    "prev_tx_dict": {
                        "hex": "02000000000101380bff9db676d159ad34849079c77e0d5c1df9c841b6a6640cba9bfc15077eea0100000000feffffff02008312000000000017a914d96bb9c5888f473dbd077d77009fb49ba2fda24287611c92a00100000017a9148722f07fbcf0fc506ea4ba9daa811d11396bbcfd870247304402202fe3c2f18e1486407bf0baabd2b3376102f0844a754d8e2fb8de71b39b3f76c702200c1fe8f7f9ef5165929ed51bf754edd7dd3e591921979cf5b891c841a1fd19d80121037c8fe1fa1ae4dfff522c532917c73c4884469e3b6a284e9a039ec612dca78eefd29c1e00",
                        "hash_hex": "4412d2a7664d01bb784a0a359e9aacf160ee436067c6a42dca355da4817ca7da",
                        "output_idx": 0,
                        "output_sats": 1213184,
                    },
                },
            ],
            "output_dicts": [
                {
                    "sats": 999500,
                    "address": "mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt",
                },
            ],
```

In [18]:
# A hex encoding of the transaction to be included as an input to the PSBT

hextx = virtual_to_spend_tx.serialize().hex()

In [19]:
input_dicts = [{
    "quorum_m": 2,
    "path_list": [
        # (xfp, root_path)
        (xfp_a, root_path),
        (xfp_b, root_path),
        (xfp_c, root_path)
    ],
    "prev_tx_dict": {
        "hex": virtual_to_spend_tx.serialize().hex(),
        "hash_hex": virtual_to_spend_tx.id(),
        "output_idx": 0,
        "output_sats": 0
    }
}]



fee = 0



### 5.3 Define Output Dicts

In [20]:
output_dicts = [{
    "sats": 0,
    # Note: we need to update this. For BIP322 the output is OP_RETURN
    "address": "3BgtPWtDCzgCbRpCbc48dgffkdbywi9Wr5",
}]


### 5.4 Create MultiSig PSBT

In [21]:
from buidl.psbt_helper import create_multisig_psbt

In [22]:
fee = 0
psbt_to_sign_bip322 = create_multisig_psbt(pubkey_records,input_dicts,output_dicts,fee)

In [27]:
base64 = psbt_to_sign_bip322.serialize_base64()


In [25]:
psbt_to_sign_bip322


Tx:

tx: f9c93111cdc7a121998f9c8c98e96cbad12dd361f875676a61a81619cba0852e
version: 1
locktime: 0
tx_ins:
f7fd427441201342a93a9252b4e95458acca7fb12aef4dd5145d22c0cd1af6a6:0
tx_outs:
0:OP_HASH160 6daaf7ccd845378a71bec802b1af09008683c2c6 OP_EQUAL 


PSBT XPUBS:
{b'O\x01\x04\x88\xb2\x1e\x04\xc3K\x1e\xd0\x00\x00\x00\x00\xcb\xed!\xb3\x19\xc4\x8a\xbbN\xb8!\x91\x9a\xfb`\xf8\xed\xe8}\x126\xbb\xe0\xc4%\xb1\x9d\x92\xa3L\xde\xc5\x02\xe7\xd7\x9f\xa5\x1f\xa5*l\xc4\xe7\xdf\xbdS\xf9\xbc\xbe<\x17&\xf1\x16k\xef*~\xca\xee$T\x80\x96\xe1\x14\xd11p^-\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00': 
HD: xpub6EyAEVH13bj6wGsMw79nTAeRhKm7d1LV3NzsUosqocvTuNi9MMk6vGNnWvFN4b1tVRjumnkawQUxMTsySnq74PcEcTnqXVT92t3wq9ztUsM
Path: d131705e:m/45'/0/0/0
, b'O\x01\x04\x88\xb2\x1e\x04_|:\x0e\x00\x00\x00\x00\xd0\x0cU*\x12\xc3\x0b\xdd4\xc8/\x9c\xec\\\x91\xac\x98&\xafF\x84\xbc\x95\x86@\xd9Hr\xad\xbc\xbf\xf5\x03\xec\xb9\x10\t\x10\xc9W\xb5\x00?\xf8R\x92&\xef\xfb(\xd5\x10\xa0\xdb\xec\x15\xf0\x89I\xe9\x14\xaa\x1c`\x

In [28]:
from buidl.psbt import PSBT

PSBT.parse_base64(base64)


Tx:

tx: 614be192b5aef7f17325ceb04179e7132822687eb34e0346ed57732a30e3bac4
version: 1
locktime: 0
tx_ins:
f7fd427441201342a93a9252b4e95458acca7fb12aef4dd5145d22c0cd1af6a6:0
tx_outs:
0:OP_RETURN 


PSBT XPUBS:
{b'\x04\x88\xb2\x1e\x04\x16_\xf6+\x00\x00\x00\x00\x84\x1f\xd7d\xe4\xd6\x08\x9dX\x06\xd9\xd8\xc4\xa1\xd5\xc8q+&\xae/\xa9\x1f(jV\xd8\x9fr(\xf4]\x03\x05\x8b\xbc\x87d\xb2<K\x00z6\xd2\x9d\xcd\xf0\n\xf8\xc3\xdb\xadr\xa5\x88\x8eu8\x0f\x15\xd3PSx': 
HD: xpub6DhSeTsokQiYjDHcqZwE8iTrj9BYuDoLeXJPhAieoNuTt3sr74kELp4CDMJ6s3RRLciTs3goxP89yJadMSDsazHQHb7sVB9uQyoxUAkqMvE
Path: e0af9290:m/45'/0/0/0
, b'\x04\x88\xb2\x1e\x04_|:\x0e\x00\x00\x00\x00\xd0\x0cU*\x12\xc3\x0b\xdd4\xc8/\x9c\xec\\\x91\xac\x98&\xafF\x84\xbc\x95\x86@\xd9Hr\xad\xbc\xbf\xf5\x03\xec\xb9\x10\t\x10\xc9W\xb5\x00?\xf8R\x92&\xef\xfb(\xd5\x10\xa0\xdb\xec\x15\xf0\x89I\xe9\x14\xaa\x1c`\x00': 
HD: xpub6EEcNjCVVKJedKvLsWJD8zNDuNXfzMxHZc4R71qwVPBjh3ByZYSeDsCQnVcpjMhWzi9iuqVaBYX2WiucFyXUxc7oYzeiAwpbnJq1jDimhQN
Path: 80e54fd9:m/45'/0/0/0
, b'

### 5.5 Update the output of the Transaction so that the output script_pubkey is OP_RETURN as per BIP322

In [26]:
psbt_to_sign_bip322.tx_obj.tx_outs[0].script_pubkey = Script([106])

## 6. Sign the PSBT to Satisfy MultiSig

In [24]:
psbt_to_sign_bip322.sign(hd_root_b)

True

In [25]:
psbt_to_sign_bip322.sign(hd_root_a)

True

In [26]:
psbt_to_sign_bip322.psbt_ins[0].sigs

{b'\x03\xec\xb9\x10\t\x10\xc9W\xb5\x00?\xf8R\x92&\xef\xfb(\xd5\x10\xa0\xdb\xec\x15\xf0\x89I\xe9\x14\xaa\x1c`\x00': b'0D\x02 4\xc4\xc5\xeb#IU\xfbU\xbel/\xfa\xe5\xaa\xd8\xeb\x16\x02U1\xe8D\xa5\xa0\x91\x9e\xcee\x1c\xee\xa5\x02 cO\x9d\xb7\xb5q\xdb\x9a\xc7\x93n\xbe\xae\xa3zl\xab\x9fbO\xfd="\xe3\xd7\xd63\xc7\xa0>RJ\x01',
 b'\x02\xe7\xd7\x9f\xa5\x1f\xa5*l\xc4\xe7\xdf\xbdS\xf9\xbc\xbe<\x17&\xf1\x16k\xef*~\xca\xee$T\x80\x96\xe1': b'0D\x02 Y\\\x15\xc1\xb2\xd1\x14\xcd\xf1_\xd7t\xf7<\xc5<$\xae\xebk\xae\x13\x86\xa7\x82Q\x9e\xcc\xd3\xa8\xc4\xe8\x02 Yr\xf2\xcc\xea\xcd\xd5\x91\xe6\x17\x1c8-}\xdc\x06\xafA\xa9\xe4\x18\x1b\xb6p\xc4\xe2\xd8.AL0b\x01'}

## Finalize the PSBT

This populates the script_sig on the tx inputs of the PSBT if possible

In [27]:
psbt_to_sign_bip322.finalize()

In [28]:
psbt_to_sign_bip322.psbt_ins[0].script_sig

OP_0 30440220595c15c1b2d114cdf15fd774f73cc53c24aeeb6bae1386a782519eccd3a8c4e802205972f2cceacdd591e6171c382d7ddc06af41a9e4181bb670c4e2d82e414c306201 3044022034c4c5eb234955fb55be6c2ffae5aad8eb16025531e844a5a0919ece651ceea50220634f9db7b571db9ac7936ebeaea37a6cab9f624ffd3d22e3d7d633c7a03e524a01 522102e7d79fa51fa52a6cc4e7dfbd53f9bcbe3c1726f1166bef2a7ecaee24548096e12103ecb9100910c957b5003ff8529226effb28d510a0dbec15f08949e914aa1c60002103058bbc8764b23c4b007a36d29dcdf00af8c3dbad72a5888e75380f15d350537853ae 

In [29]:
# Note: the script_sig for the input on the Tx object is not yet set

psbt_to_sign_bip322.tx_obj.tx_ins[0].script_sig

## 7. Generate Final Tx able to be spent on network

### Note: final_tx() throws error, because we set no fee. BUT for BIP322 we do not need a fee.

```python
    def final_tx(self):
        """Returns the broadcast-able transaction"""
        # clone the transaction from self.tx_obj
        tx_obj = self.tx_obj.clone()
        # determine if the transaction is segwit by looking for a witness field
        #  in any PSBTIn. if so, set tx_obj.segwit = True
        if any([psbt_in.witness for psbt_in in self.psbt_ins]):
            tx_obj.segwit = True
        # iterate through the transaction and PSBT inputs together
        #  using zip(tx_obj.tx_ins, self.psbt_ins)
        for tx_in, psbt_in in zip(tx_obj.tx_ins, self.psbt_ins):
            # set the ScriptSig of the transaction input
            tx_in.script_sig = psbt_in.script_sig
            # Exercise 7: if the tx is segwit, set the witness as well
            if tx_obj.segwit:
                # witness should be the PSBTIn witness or an empty Witness()
                tx_in.witness = psbt_in.witness or Witness()
        # check to see that the transaction verifies
        if not tx_obj.verify():
            raise RuntimeError("transaction invalid")
        # return the now filled in transaction
        return tx_obj
```

In [30]:
# THIS WILL FAIL
psbt_to_sign_bip322.final_tx()

This transaction won't relay without having a fee of at least 313


RuntimeError: transaction invalid

### So lets rewrite that fn

In [31]:
def final_bip322_tx(psbt_obj):
    """Returns the broadcast-able transaction"""
    # clone the transaction from self.tx_obj
    tx_obj = psbt_obj.tx_obj.clone()
    # determine if the transaction is segwit by looking for a witness field
    #  in any PSBTIn. if so, set tx_obj.segwit = True
    if any([psbt_in.witness for psbt_in in psbt_obj.psbt_ins]):
        tx_obj.segwit = True
    # iterate through the transaction and PSBT inputs together
    #  using zip(tx_obj.tx_ins, self.psbt_ins)
    for tx_in, psbt_in in zip(tx_obj.tx_ins, psbt_obj.psbt_ins):
        # set the ScriptSig of the transaction input
        tx_in.script_sig = psbt_in.script_sig
        print(tx_in.script_sig)
        # Exercise 7: if the tx is segwit, set the witness as well
        if tx_obj.segwit:
            # witness should be the PSBTIn witness or an empty Witness()
            tx_in.witness = psbt_in.witness or Witness()
    # check to see that the transaction verifies MINUS FEE CHECK
    for i in range(len(tx_obj.tx_ins)):
        if not tx_obj.verify_input(i):
            raise RuntimeError("transaction invalid")
#     if not tx_obj.verify():
#         raise RuntimeError("transaction invalid")
    # return the now filled in transaction
    return tx_obj

In [32]:
finalized_to_sign_tx = final_bip322_tx(psbt_to_sign_bip322)

OP_0 30440220595c15c1b2d114cdf15fd774f73cc53c24aeeb6bae1386a782519eccd3a8c4e802205972f2cceacdd591e6171c382d7ddc06af41a9e4181bb670c4e2d82e414c306201 3044022034c4c5eb234955fb55be6c2ffae5aad8eb16025531e844a5a0919ece651ceea50220634f9db7b571db9ac7936ebeaea37a6cab9f624ffd3d22e3d7d633c7a03e524a01 522102e7d79fa51fa52a6cc4e7dfbd53f9bcbe3c1726f1166bef2a7ecaee24548096e12103ecb9100910c957b5003ff8529226effb28d510a0dbec15f08949e914aa1c60002103058bbc8764b23c4b007a36d29dcdf00af8c3dbad72a5888e75380f15d350537853ae 


In [33]:
finalized_to_sign_tx.tx_outs[0].script_pubkey

OP_RETURN 

In [34]:
finalized_to_sign_tx.tx_ins[0].script_sig

OP_0 30440220595c15c1b2d114cdf15fd774f73cc53c24aeeb6bae1386a782519eccd3a8c4e802205972f2cceacdd591e6171c382d7ddc06af41a9e4181bb670c4e2d82e414c306201 3044022034c4c5eb234955fb55be6c2ffae5aad8eb16025531e844a5a0919ece651ceea50220634f9db7b571db9ac7936ebeaea37a6cab9f624ffd3d22e3d7d633c7a03e524a01 522102e7d79fa51fa52a6cc4e7dfbd53f9bcbe3c1726f1166bef2a7ecaee24548096e12103ecb9100910c957b5003ff8529226effb28d510a0dbec15f08949e914aa1c60002103058bbc8764b23c4b007a36d29dcdf00af8c3dbad72a5888e75380f15d350537853ae 

In [35]:
finalized_to_sign_tx.id()

'10cf02c00a5c350238c265dc44e4e402c1802cee71eecbdacaaf859558dbe07f'

In [37]:
# Double check the single tx_input is valid
finalized_to_sign_tx.verify_input(0)

True

## 8. Serialize and encode to_sign Transaction as BIP322 Signature

In [38]:
from buidl.helper import base64_encode
bip322_musig = base64_encode(finalized_to_sign_tx.serialize())

In [39]:
bip322_musig

'AQAAAAGm9hrNwCJdFNVN7yqxf8qsWFTptFKSOqlCEyBBdEL99wAAAAD8AEcwRAIgWVwVwbLRFM3xX9d09zzFPCSu62uuE4anglGezNOoxOgCIFly8szqzdWR5hccOC193AavQankGBu2cMTi2C5BTDBiAUcwRAIgNMTF6yNJVftVvmwv+uWq2OsWAlUx6ESloJGezmUc7qUCIGNPnbe1cduax5Nuvq6jemyrn2JP/T0i49fWM8egPlJKAUxpUiEC59efpR+lKmzE59+9U/m8vjwXJvEWa+8qfsruJFSAluEhA+y5EAkQyVe1AD/4UpIm7/so1RCg2+wV8IlJ6RSqHGAAIQMFi7yHZLI8SwB6NtKdzfAK+MPbrXKliI51OA8V01BTeFOu/////wEAAAAAAAAAAAFqAAAAAA=='

## 9. Verify The Produced BIP322 Signature

In [40]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd


In [41]:
signing_address = p2sh_address
bip322_sig = bip322_musig
message = "Hello World"


In [42]:
from src.message import verify_message

In [43]:

verification_result = verify_message(signing_address, bip322_sig, message)

print("Signature is verified : ", verification_result)

Signature is verified :  True


In [46]:
# This is the p2sh address that the to_spend tx was locked with.
finalized_to_sign_tx.tx_ins[0].script_pubkey().address()

'3ChoQmPrnmuuyW3tebfrMqiZgfXYRvXb99'